<a href="https://colab.research.google.com/github/varunyadav17/Credit-Card-Fraud/blob/master/SkinCancerPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyunpack patool
import os
import gc
import json
import shutil
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

from pyunpack import Archive

from keras import backend as K
from keras.models import Sequential
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
Archive('/content/drive/My Drive/Data.rar').extractall('.')

**Preprocessing**

In [0]:
count = len(os.listdir('Data'))

In [0]:
INPUT_DESC_PATH = 'Data/Descriptions/'
INPUT_IMAGE_PATH = 'Data/Images'
if count > 0 and os.path.exists(INPUT_DESC_PATH):
  descfiles = os.listdir(INPUT_DESC_PATH)

In [0]:
#descfiles = [i[:-5] for i in descfiles if i.endswith('.json')]

In [0]:
image_types = {}
if len(descfiles) > 0:
  for i in range(len(descfiles)):
    file = os.path.join(INPUT_DESC_PATH, descfiles[i])
    with open(file) as json_file:
      data = json.load(json_file)
      if descfiles[i].endswith('.json'):
        key = descfiles[i][:-5]
      else:
        key = descfiles[i]
      image_types[key] = data['meta']['clinical']['benign_malignant']

**Create Directory Tree**

In [0]:
if not os.path.exists('Train'):
  os.mkdir('Train')
  os.chdir('Train')
  os.mkdir('Benign')
  os.mkdir('Malignant')
  
if not os.path.exists('Test'):
  os.chdir('..')
  os.mkdir('Test') 
  os.chdir('Test')
  os.mkdir('Benign')
  os.mkdir('Malignant')
  
if not os.path.exists('Val'):
  os.chdir('..')
  os.mkdir('Val')
  os.chdir('Val')
  os.mkdir('Benign')
  os.mkdir('Malignant')
  os.chdir('..')

In [0]:
benign_images = [key for (key, value) in image_types.items() if value == 'benign']

In [0]:
# Append extensions to files
complete_benign_files = []
input_files = os.listdir(INPUT_IMAGE_PATH)
for i in range(len(benign_images)):
  newfile = [file for file in input_files if file.startswith(benign_images[i])]
  complete_benign_files.append(newfile)

In [0]:
malignant_images = [key for (key, value) in image_types.items() if value == 'malignant']

In [0]:
# Append extensions to files
complete_malignant_files = []
input_files = os.listdir(INPUT_IMAGE_PATH)
for i in range(len(malignant_images)):
  newfile = [file for file in input_files if file.startswith(malignant_images[i])]
  complete_malignant_files.append(newfile)

In [0]:
train, val, test = np.split(pd.DataFrame(complete_benign_files).sample(frac = 1) ,[int(.6 * len(complete_benign_files)), int(.8 * len(complete_benign_files))])

In [0]:
# Move benign files to their respective directories
train_list = train[0].tolist()
for i in range(len(train)):
  src = os.path.join(INPUT_IMAGE_PATH, (train_list[i]))
  dest = os.path.join('Train/Benign', train_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)
    
test_list = test[0].tolist()
for i in range(len(test)):
  src = os.path.join(INPUT_IMAGE_PATH, (test_list[i]))
  dest = os.path.join('Test/Benign', test_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

val_list = val[0].tolist()
for i in range(len(val)):
  src = os.path.join(INPUT_IMAGE_PATH, (val_list[i]))
  dest = os.path.join('Val/Benign', val_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

In [0]:
# Now add malignant files in similar ratio to Train,Test and Val directories
train, val, test = np.split(pd.DataFrame(complete_malignant_files).sample(frac = 1) ,[int(.6 * len(complete_malignant_files)), int(.8 * len(complete_malignant_files))])

In [0]:
# Move malignant files to their respective directories
train_list = train[0].tolist()
for i in range(len(train)):
  src = os.path.join(INPUT_IMAGE_PATH, (train_list[i]))
  dest = os.path.join('Train/Malignant', train_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)
    
test_list = test[0].tolist()
for i in range(len(test)):
  src = os.path.join(INPUT_IMAGE_PATH, (test_list[i]))
  dest = os.path.join('Test/Malignant', test_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

val_list = val[0].tolist()
for i in range(len(val)):
  src = os.path.join(INPUT_IMAGE_PATH, (val_list[i]))
  dest = os.path.join('Val/Malignant', val_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

In [0]:
shutil.rmtree('Data')
#os.remove('TestData.rar')

**Callbacks**

In [0]:
batch_size = 2
if not os.path.exists('Output'):
  os.mkdir('Output')
  os.chdir('Output')
  os.mkdir('Models')
  os.mkdir('Logs')
  os.chdir('..')
model_dir = 'Output/Models/'
model_file = model_dir + 'Models' + '_{}'.format(datetime.datetime.now())+'.hdf5'
log_dir = 'Output/Logs'

In [0]:
tf.reset_default_graph()
K.clear_session()
gc.collect()
  
checkpoint = None
reduce_lr = None
early_stopping = None
tensorboard = None

In [0]:
checkpoint = ModelCheckpoint(model_file, monitor = 'val_acc', save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1, restore_best_weights = True)
tensorboard = TensorBoard(log_dir = log_dir, batch_size = batch_size, update_freq = 'batch')
reduce_lr = ReduceLROnPlateau(monitor= 'val_loss', patience = 5, cooldown = 2, lr = 0.0000000001, verbose = 1)

callbacks = [checkpoint, early_stopping, tensorboard, reduce_lr]

**Model Building**

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (32, 32, 3)))
#model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

#model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
#model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
#model.add(MaxPooling2D(pool_size = (2, 2)))

#model.add(Conv2D(96, (3, 3), activation = 'relu', padding = 'same'))
#model.add(Conv2D(96, (3, 3), activation = 'relu', padding = 'same'))
#model.add(MaxPooling2D(pool_size = (2, 2)))

#model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
#model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
          
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid'))

model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 1,049,858
Trainable params: 1,049,858
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
class_mode = 'categorical'
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = train_datagen.flow_from_directory('Train', target_size = (32, 32), batch_size = 32, class_mode = class_mode, shuffle = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory('Test', target_size = (32, 32), batch_size = 32, class_mode = class_mode, shuffle = False)

val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = val_datagen.flow_from_directory('Val', target_size = (32, 32), batch_size = 32, class_mode = class_mode, shuffle = False)

Found 840 images belonging to 2 classes.
Found 280 images belonging to 2 classes.
Found 280 images belonging to 2 classes.


In [0]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)

In [0]:
class_weight

array([0.7 , 1.75])

In [0]:
model.fit_generator(train_generator, steps_per_epoch = len(train_generator), epochs = 100, verbose = 2, validation_data = val_generator, validation_steps = len(val_generator), 
                    callbacks = callbacks, class_weight = class_weight)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
 - 75s - loss: 0.5852 - acc: 0.7219 - val_loss: 0.7489 - val_acc: 0.7143
Epoch 2/100
 - 74s - loss: 0.5957 - acc: 0.7152 - val_loss: 0.5381 - val_acc: 0.7143
Epoch 3/100
 - 74s - loss: 0.5379 - acc: 0.7152 - val_loss: 0.5275 - val_acc: 0.7179
Epoch 4/100
 - 74s - loss: 0.4904 - acc: 0.7152 - val_loss: 0.4888 - val_acc: 0.7357
Epoch 5/100
 - 74s - loss: 0.4569 - acc: 0.7639 - val_loss: 0.4719 - val_acc: 0.7357
Epoch 6/100
 - 73s - loss: 0.4652 - acc: 0.7351 - val_loss: 0.4571 - val_acc: 0.7464
Epoch 7/100
 - 73s - loss: 0.4268 - acc: 0.8054 - val_loss: 0.4882 - val_acc: 0.7286
Epoch 8/100
 - 73s - loss: 0.4218 - acc: 0.7709 - val_loss: 0.4267 - val_acc: 0.7607
Epoch 9/100
 - 73s - loss: 0.3967 - acc: 0.7953 - val_loss: 0.4158 - val_acc: 0.7893
Epoch 10/100
 - 73s - loss: 0.3990 - acc: 0.8114 - val_loss: 0.4039 - val_acc: 0.8036
Epoch 11/100
 - 73s - loss: 0.3737 - acc: 0.8297 - va

In [0]:
from keras.models import load_model
model_file = 'Output/Models/' + str([file for file in os.listdir('Output/Models/') if file.startswith('Models')][0])
model = load_model(model_file)

In [0]:
results = model.evaluate_generator(test_generator, steps = len(test_generator), verbose = 1)

9/9 [==============================] - 20s 2s/step


In [0]:
print("Loss = %s" %results[0])
print("Accuracy = %s" %results[1])

Loss = 0.33216618895530703
Accuracy = 0.8464285731315613


In [0]:
y_pred = model.predict_generator(train_generator, len(train_generator), verbose = 1)
y_pred = y_pred.argmax(axis = -1)
y_true = train_generator.classes

27/27 [==============================] - 61s 2s/step


In [0]:
precision = precision_score(y_pred, y_true)
recall = recall_score(y_true, y_pred)
f1_score = f1_score(y_true, y_pred)

In [0]:
print("Precision = %s Recall = %s F1 Score = %s" % (precision, recall, f1_score))

Precision = 0.22083333333333333 Recall = 0.22083333333333333 F1 Score = 0.2610837438423645


In [0]:
numofbatch = len(test_generator)
batch_no = random.randint(0, numofbatch - 1)
y_img_batch, y_true_batch = test_generator[batch_no]
y_true_batch = y_true_batch.argmax(axis = -1)

y_pred_batch = model.predict(y_img_batch)
y_pred_batch = y_pred_batch.argmax(axis = -1)

sizeofbatch = len(y_true_batch)

In [0]:
type(y_pred_batch)

numpy.ndarray

In [0]:
batch_size = len(y_pred_batch)
accuracy = str(np.mean(y_true == y_pred) * 100) + " %"
print("Batch no = %s Batch size = %s Accuracy = %s" % (batch_no, batch_size, accuracy))

Batch no = 1 Batch size = 32 Accuracy = 64.28571428571429 %


In [0]:
y_true_batch

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
y_pred_batch

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [0]:
test_generator.class_indices.items()

dict_items([('Benign', 0), ('Malignant', 1)])